In [30]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Primero generamos el rango de datos horarios para el año 2021 y después aplicar la lógica del precio para 2.0 TD.
fechas = pd.date_range(start="2021-01-01 00:00", end="2021-12-31 23:00", freq='H')
df = pd.DataFrame({'datetime': fechas})
df['hour'] = df['datetime'].dt.hour
df['month'] = df['datetime'].dt.month
df['weekday'] = df['datetime'].dt.weekday  # En orden, 0 se asignaría al lunes y 6 se correspondería al domingo.

# Simulamos una demanda de carga de la casa realista; a la noche se demandará más energía que a la mañana.
def demanda_realista(hour):
    # En un principio se supuso una demanda hiper realista que abarcaba desde 500 W hasta los 6.5 kW. Sin embargo, 
    # la producción de las fuentes renovables (nuestros modelos) no es tan grande como para abastecer una demanda hiper realista
    # de un inmueble, y es por ello que se ha adaptado para lograr resultados en todos los apartados más lógicos.
    if 20 <= hour or hour < 5:
        return np.random.uniform(700, 1100)  # Demanda noche. 
    elif 12 <= hour < 20:
        return np.random.uniform(300, 500)  # Demanda de tarde.
    else:
        return np.random.uniform(100, 300)  # Demanda a la mañana/mediodía.

df['demanda'] = df['hour'].apply(demanda_realista) # Nuestra columna con la carga de demanda de Watts por hora.

# El siguiente paso constará en asignar el periodo horario según lo establecido en el BOE para el peaje 2.0 TD
def asignar_periodo(row):
    if row['weekday'] >= 5:
        return 'P3'
    if row['month'] in [6, 7, 8, 9]:  # Periodo de verano.
        if 10 <= row['hour'] < 16:
            return 'P1'
        elif 8 <= row['hour'] < 10 or 16 <= row['hour'] < 24:
            return 'P2'
        else:
            return 'P3'
    else:  # Resto del año.
        if 18 <= row['hour'] < 22:
            return 'P1'
        elif 8 <= row['hour'] < 18 or 22 <= row['hour'] < 24:
            return 'P2'
        else:
            return 'P3'

df['periodo'] = df.apply(asignar_periodo, axis=1)

# Para cada periodo que hemos designado, asignamos los precios que le corresponde a cada uno.
precios = {'P1': 0.027787/1000, 'P2': 0.019146/1000, 'P3': 0.000703/1000} # €/Wh.
df['precio_eur_Wh'] = df['periodo'].map(precios) # Este será nuestra columna con los precios horarios de la energía.

# # Paso 5: Calcular coste horario
# df['coste_eur'] = df['demanda'] * df['precio_eur_kWh']

# # Resultado final
# df_final = df[['datetime', 'demanda', 'periodo', 'precio_eur_kWh', 'coste_eur']]

# # (Opcional) Guardar a CSV
# # df_final.to_csv("consumo_casa_2021.csv", index=False)

# print(df_final.head())
# print(len(df_final))

C:\Users\jonnu\AppData\Local\Temp\ipykernel_23908\3393824449.py:7: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  fechas = pd.date_range(start="2021-01-01 00:00", end="2021-12-31 23:00", freq='H')


In [31]:
# Cargamos el archivo que contiene todas las mediciones necesarías para los agentes eólicos y solares.
data = pd.read_csv("data/data.csv")
print(len(data))

# Hacemos un append con las dos columnas que queremos añadir (precio y demanda (W)).
data['precio_eur_Wh'] = df['precio_eur_Wh'].values
data['demanda_W'] = df['demanda'].values

# Guardamos el nuevo archivo preparado con todo.
data.to_csv("data/data_complete.csv")

8760
